# Import module

In [46]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers, applications

Directory of dog vs cat

In [24]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/Users/user/Desktop/python/siraj/dogvscat/train'
validation_data_dir = '/Users/user/Desktop/python/siraj/dogvscat/validation'

# Create dataset

In [25]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=16,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 22778 images belonging to 2 classes.
Found 2222 images belonging to 2 classes.


# Model architecture definition

In [27]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [28]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [29]:
nb_epoch = 30
nb_train_samples = 2048
nb_validation_samples = 832

In [31]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples)

Epoch 1/30
2048/2048 [==============================] - 208s - loss: 0.5777 - acc: 0.7013 - val_loss: 0.5109 - val_acc: 0.7540
Epoch 2/30
2048/2048 [==============================] - 152s - loss: 0.4859 - acc: 0.7787 - val_loss: 0.4231 - val_acc: 0.8048
Epoch 3/30
2048/2048 [==============================] - 152s - loss: 0.4531 - acc: 0.8038 - val_loss: 0.4261 - val_acc: 0.8121
Epoch 4/30
2048/2048 [==============================] - 150s - loss: 0.4357 - acc: 0.8141 - val_loss: 0.4931 - val_acc: 0.8022
Epoch 5/30
2048/2048 [==============================] - 149s - loss: 0.4183 - acc: 0.8272 - val_loss: 0.4115 - val_acc: 0.8243
Epoch 6/30
2048/2048 [==============================] - 148s - loss: 0.4082 - acc: 0.8311 - val_loss: 0.4215 - val_acc: 0.8358
Epoch 7/30
2048/2048 [==============================] - 149s - loss: 0.4158 - acc: 0.8269 - val_loss: 0.4663 - val_acc: 0.8287
Epoch 8/30
2048/2048 [==============================] - 148s - loss: 0.4034 - acc: 0.8353 - val_loss: 0.5026 - 

In [34]:
model.save_weights('models/basic_cnn_30_epochs.h5')

In [35]:
#model.load_weights('models_trained/basic_cnn_30_epochs.h5')

# Evaluating on validation set
Computing loss and accuracy

In [36]:
model.evaluate_generator(validation_generator, nb_validation_samples)

[0.53690823765334417, 0.76682055563079699]

# Data augmentation for improving the model
shearing, zooming and flipping

In [37]:
train_datagen_augmented = ImageDataGenerator(
        rescale=1./255,        # normalize pixel values to [0,1]
        shear_range=0.2,       # randomly applies shearing transformation
        zoom_range=0.2,        # randomly applies shearing transformation
        horizontal_flip=True)  # randomly flip the images

# same code as before
train_generator_augmented = train_datagen_augmented.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 22778 images belonging to 2 classes.


In [38]:
model.fit_generator(
        train_generator_augmented,
        steps_per_epoch=nb_train_samples,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples)

Epoch 1/30
  42/2048 [..............................] - ETA: 379s - loss: 0.6349 - acc: 0.7083

KeyboardInterrupt: 

In [ ]:
model.save_weights('models/augmented_30_epochs.h5')

In [ ]:
#model.load_weights('models_trained/augmented_30_epochs.h5')

# Evaluating on validation set

In [ ]:
model.evaluate_generator(validation_generator, nb_validation_samples)

# Using a pre-trained model

VGG16, a model trained on the ImageNet dataset - which contains millions of images classified in 1000 categories.
On top of it, we add a small multi-layer perceptron and we train it on our dataset.

In [86]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_1")`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_2")`
  """
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_1")`
  if __name__ == '__main__':
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_2")`
  # This is added back by InteractiveShellApp.init_path()
C:\Users\user\Anaconda3\envs\t

AttributeError: module 'keras.models' has no attribute 'load_weights'

# Using the VGG16 model to process samples

In [80]:
# prepare data augmentation configuration  . . . do we need this?
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=32,
        class_mode='binary')

Found 22778 images belonging to 2 classes.
Found 2222 images belonging to 2 classes.


In [81]:
model.compile(loss='binary_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [82]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [85]:
nb_epoch = 5
model.fit_generator(
    train_generator,
    samples_per_epoch=10000,
    epochs=nb_epoch,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., epochs=5, validation_steps=832, steps_per_epoch=312)`
  import sys


Epoch 1/5


ValueError: Error when checking model target: expected dense_16 to have shape (None, 10) but got array with shape (32, 1)